In [ ]:
pip install langchain_community langchain_chroma pymupdf

In [83]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문에 대한 답변을 제공하는 도우미입니다. 모든 답변은 한국어로 친절하게 대답하세요."),
    ("user", "{input}"),
])

chain = prompt | llm | StrOutputParser()


In [ ]:
chain.invoke({ "input"  : "Stock Buyback의 정의와 예시에 대해 알려줘" })

# Document Loader
[https://python.langchain.com/api_reference/community/document_loaders.html#]

1. CharacterTextSplitter
- 분할 기준: 지정된 단일 문자(예: \n, 공백 등)를 기준으로 텍스트를 분할합니다.​
- 특징:
단순한 문자 기반 분할 방식으로, 구조화되지 않은 텍스트에 적합합니다.​
문맥이나 의미를 고려하지 않기 때문에, 문장이나 단어가 중간에 잘릴 수 있습니다.

2. RecursiveCharacterTextSplitter
- 분할 기준: 여러 구분자(기본값: ["\n\n", "\n", " ", ""])를 우선순위에 따라 재귀적으로 적용하여 텍스트를 분할합니다.​
- 특징:
문단 → 문장 → 단어 순으로 분할을 시도하여, 텍스트의 의미와 문맥을 최대한 보존합니다.​
구조화된 텍스트나 자연어 처리에서 의미 단위를 유지하려는 경우에 적합합니다.

3. TokenTextSplitter
- 분할 기준: 토큰 수를 기준으로 텍스트를 분할합니다.​
- 특징: LLM의 토큰 제한을 고려하여 텍스트를 분할하므로, 모델 입력에 최적화된 형태로 텍스트를 준비할 수 있습니다.​ 언어별 토크나이저를 활용하여 정확한 토큰 단위 분할이 가능합니다


In [ ]:
with open("data/finance-keywords.txt", "r", encoding="utf-8") as f:
    file = f.read()
    print(len(file))

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("data/finance-keywords.txt", encoding="utf-8")
documents = loader.load()

text_splitter1 = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = loader.load_and_split(text_splitter1)

len(chunks)

In [ ]:
chunks

In [ ]:
chunks[0].metadata

In [ ]:
chunks[0].page_content

# VectorStore 생성

## Chroma
 - Langchain Chroma 문서[https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/]
 - Langchain VectorStore 문서[https://python.langchain.com/v0.2/docs/integrations/vectorstores/]

### 벡터 저장소 생성 (from_documents)
from_documents 클래스 메서드는 문서 리스트로부터 벡터 저장소를 생성합니다.

### 매개변수

- documents (List[Document]): 벡터 저장소에 추가할 문서 리스트
- embedding (Optional[Embeddings]): 임베딩 함수. 기본값은 None
- ids (Optional[List[str]]): 문서 ID 리스트. 기본값은 None
- collection_name (str): 생성할 컬렉션 이름.
- persist_directory (Optional[str]): 컬렉션을 저장할 디렉토리. 기본값은 None
- client_settings (Optional[chromadb.config.Settings]): Chroma 클라이언트 설정
- client (Optional[chromadb.Client]): Chroma 클라이언트 인스턴스
- collection_metadata (Optional[Dict]): 컬렉션 구성 정보. 기본값은 None

### 참고

- persist_directory가 지정되면 컬렉션이 해당 디렉토리에 저장됩니다. 지정되지 않으면 데이터는 메모리에 임시로 저장됩니다.
- 이 메서드는 내부적으로 from_texts 메서드를 호출하여 벡터 저장소를 생성합니다.
- 문서의 page_content는 텍스트로, metadata는 메타데이터로 사용됩니다.

### 반환값
- Chroma: 생성된 Chroma 벡터 저장소 인스턴스


생성시 documents 매개변수로 Document 리스트를 전달합니다. embedding 에 활용할 임베딩 모델을 지정하며, namespace 의 역할을 하는 collection_name 을 지정할 수 있습니다.


In [24]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
embeded = embeddings.embed_query("비씨카드1")


In [ ]:
len(embeded)

In [63]:

DB_PATH = "./chroma_db"
db = Chroma.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    persist_directory=DB_PATH,
    collection_name="finance-keywords",
)


In [64]:
doc_results = db.similarity_search(
    query="Stock Buyback의 정의와 예시에 대해 알려줘",
    k=3,
    filter={'source':'data/finance-keywords.txt'}
)

```python
    db.get(where={"source":"data/finance-keywords.txt"})
    db.get(['1','2'])
    db.delete(ids=["1"])
    db.reset_collection()
```

In [ ]:
db.get(limit=4, where={"source":"data/finance-keywords.txt"})

In [ ]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k" : 2,
        "score_threshold" : 0.7,
    }
)
docs = retriever.invoke("Stock Buyback")

for doc in docs :
    print(doc.page_content)
    print("=" * 50)

In [ ]:
config = {
    "configurable" : {
        "search_type" : "similarity_score_threshold" ,
        "search_kwargs" : {
            "score_threshold" : 0.7,
        }
    }
}
docs = retriever.invoke("Stock Buyback에 대한 정의와 예시를 알려주세요", config=config)
for doc in docs : 
    print(doc)
    print("=" * 50)

In [19]:
def format_docs(docs) :
    return "\n\n".join([f"문서내용:{doc.page_content}, 출처:{doc.metadata['source']}" for doc in docs])

In [84]:
from langchain_core.runnables import RunnablePassthrough
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 친절한 AI 어시스턴트 입니다. 주어진 문서의 내용에 따라 충실히 답변하세요.'),
    ('system', '주어진 문서 : {documents}'),
    ('user', '질문 : {question}')
])

chain = ({ 'documents' : retriever | format_docs , "question" : RunnablePassthrough() }
    | prompt 
    | llm
    | StrOutputParser()
)


In [ ]:
chain.invoke("Stock Buyback에 대한 정의와 예시를 알려주세요")

In [ ]:
chain.invoke("비씨카드의 2024년에 개최된 이사회는 몇번이고, 이사의 참석률은 몇퍼센트 인가요?")

In [21]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser

# Step1 : 문서 로드
loader = PyMuPDFLoader('data/bccard.pdf')
docs = loader.load()

# Step2 : 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

In [ ]:
print(len(split_documents))
print("=" *50)
print(split_documents[10].page_content)
print("=" *50)
print(split_documents[10].metadata)

In [31]:
def format_docs(docs) :
    return "\n\n".join([f"문서내용:{doc.page_content}, 출처:{doc.metadata['source']}/{doc.metadata['page']}" for doc in docs])

In [ ]:
# Step3 : 임베딩 & VectorDB 생성
vectorstore = Chroma.from_documents(documents=split_documents, 
                                    embedding=OpenAIEmbeddings())


In [32]:
# Step4 : 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

# Step5 : 프롬프트 생성
prompt = ChatPromptTemplate.from_messages([
    ('system', '당신은 친절한 AI 어시스턴트 입니다. 주어진 문서의 내용에 따라 충실히 답변하세요.'),
    ('system', '답변 시 출처 정보도 알려주세요. Example) 답변 (출처:~~)'),
    ('system', '주어진 문서 : {documents}'),
    ('user', '질문 : {question}')
])

# Step6 : LLM 및 Chain 생성

llm = ChatOpenAI(model="gpt-4o-mini")

chain = ({ 'documents' : retriever | format_docs , "question" : RunnablePassthrough() }
    | prompt 
    | llm
    | StrOutputParser()
)


In [ ]:
chain.invoke("비씨카드의 2024년에 개최된 이사회는 몇번인가요?")